In [5]:
!pip install shapely
!pip install pandas
!pip install cffi

!pip install --pre --force-reinstall --no-deps pymeos pymeos_cffi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 2.8 MB/s eta 0:00:0000:0100:01


In [1]:
import pandas as pd
from pymeos import *

pymeos_initialize()


/tmp/ipykernel_12245/3447813113.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

ais = pd.read_csv(
    "aisdk-2023-08-01.zip",
    usecols=["# Timestamp", "MMSI", "Latitude", "Longitude", "SOG"], nrows=10000
)
ais.columns = ["t", "mmsi", "lat", "lon", "sog"]


In [3]:
ais

,t,mmsi,lat,lon,sog
0,01/08/2023 00:00:00,219020187,56.795448,8.864770,0.0
1,01/08/2023 00:00:00,219000873,56.990862,10.304587,0.0
2,01/08/2023 00:00:00,219008746,56.609180,10.300487,0.0
3,01/08/2023 00:00:00,244061183,91.000000,0.000000,NaN
4,01/08/2023 00:00:00,219005496,57.754300,10.151212,0.2
...,...,...,...,...,...
9995,01/08/2023 00:01:24,219021235,57.120703,8.599565,0.0
9996,01/08/2023 00:01:24,219005477,56.152500,10.251267,0.1
9997,01/08/2023 00:01:25,241831000,56.593500,8.118667,0.1
9998,01/08/2023 00:01:25,231794000,56.794665,10.857220,6.7


ais = ais[ais["mmsi"] == 219020187]
ais = ais[ais["t"] != 0]

In [4]:

ais["t"] = pd.to_datetime(ais["t"], format='%d/%m/%Y %H:%M:%S')
ais = ais[ais["mmsi"] != 0]
ais = ais.drop_duplicates(["t", "mmsi"])
ais = ais[(ais["lat"] >= 40.18) & (ais["lat"] <= 84.17)]
ais = ais[(ais["lon"] >= -16.1) & (ais["lon"] <= 32.88)]
ais = ais[(ais["sog"] >= 0) & (ais["sog"] <= 1022)]
ais.dropna()
ais.head()



,t,mmsi,lat,lon,sog
0,2023-08-01,219020187,56.795448,8.864770,0.0
1,2023-08-01,219000873,56.990862,10.304587,0.0
2,2023-08-01,219008746,56.609180,10.300487,0.0
4,2023-08-01,219005496,57.754300,10.151212,0.2
5,2023-08-01,219014072,54.918928,9.597852,0.0


In [5]:
ais["instant"] = ais.apply(lambda row: TGeogPointInst(point=(row["lon"], row["lat"]), timestamp=row["t"]),axis=1)

# > 5 min, reduce datasize 

In [9]:
ais.iloc[0]

t                                      2023-08-01 00:00:00
mmsi                                             219020187
lat                                              56.795448
lon                                                8.86477
sog                               0@2023-08-01 00:00:00+02
instant    POINT(8.86477 56.795448)@2023-08-01 00:00:00+02
Name: 0, dtype: object

In [11]:
ais["instant"] = ais.apply(
    lambda row: TGeogPointInst(point=(row["lon"], row["lat"]), timestamp=row["t"]),
    axis=1,
)

ais.drop(["lat", "lon"], axis=1, inplace=True)
ais

,t,mmsi,sog,instant
0,2023-08-01 00:00:00,219020187,0@2023-08-01 00:00:00+02,POINT(8.86477 56.795448)@2023-08-01 00:00:00+02
1,2023-08-01 00:00:00,219000873,0@2023-08-01 00:00:00+02,POINT(10.304587 56.990862)@2023-08-01 00:00:00+02
2,2023-08-01 00:00:00,219008746,0@2023-08-01 00:00:00+02,POINT(10.300487 56.60918)@2023-08-01 00:00:00+02
4,2023-08-01 00:00:00,219005496,0.2@2023-08-01 00:00:00+02,POINT(10.151212 57.7543)@2023-08-01 00:00:00+02
5,2023-08-01 00:00:00,219014072,0@2023-08-01 00:00:00+02,POINT(9.597852 54.918928)@2023-08-01 00:00:00+02
...,...,...,...,...
9995,2023-08-01 00:01:24,219021235,0@2023-08-01 00:01:24+02,POINT(8.599565 57.120703)@2023-08-01 00:01:24+02
9996,2023-08-01 00:01:24,219005477,0.1@2023-08-01 00:01:24+02,POINT(10.251267 56.1525)@2023-08-01 00:01:24+02
9997,2023-08-01 00:01:25,241831000,0.1@2023-08-01 00:01:25+02,POINT(8.118667 56.5935)@2023-08-01 00:01:25+02
9998,2023-08-01 00:01:25,231794000,6.7@2023-08-01 00:01:25+02,POINT(10.85722 56.794665)@2023-08-01 00:01:25+02


In [12]:

trajectories = (
    ais.groupby("mmsi")
    .aggregate(
        {
            "instant": lambda x: TGeogPointSeq.from_instants(x, upper_inc=True),
            "sog": lambda x: TFloatSeq.from_instants(x, upper_inc=True),
        }
    )
    .rename({"instant": "trajectory"}, axis=1)
)
trajectories["distance"] = trajectories["trajectory"].apply(lambda t: t.length())
rows = [ais.iloc[0]]

In [13]:
trajectories

,trajectory,sog,distance
mmsi,,,
5322,[POINT(12.606042 55.684293)@2023-08-01 00:00:0...,"[0.1@2023-08-01 00:00:00+02, 0.2@2023-08-01 00...",23.883298
2190045,[POINT(8.423332 55.47179)@2023-08-01 00:00:07+...,"[0@2023-08-01 00:00:07+02, 0@2023-08-01 00:01:...",1.416454
109080297,[POINT(9.916937 57.815327)@2023-08-01 00:00:15...,"[1.9@2023-08-01 00:00:15+02, 1.9@2023-08-01 00...",0.000000
109080632,[POINT(7.840475 56.994502)@2023-08-01 00:00:59...,[0.9@2023-08-01 00:00:59+02],0.000000
205576930,[POINT(9.749615 54.971258)@2023-08-01 00:01:12...,[0.1@2023-08-01 00:01:12+02],0.000000
...,...,...,...
636023064,[POINT(8.830263 57.759765)@2023-08-01 00:00:01...,"[7.1@2023-08-01 00:00:01+02, 7.1@2023-08-01 00...",298.346515
636092635,[POINT(10.236577 56.15168)@2023-08-01 00:01:07...,[0@2023-08-01 00:01:07+02],0.000000
671057300,[POINT(10.813455 55.298015)@2023-08-01 00:00:4...,[0@2023-08-01 00:00:46+02],0.000000


In [20]:
# create a datetime from this string : 2023-08-01 00:01:24
from datetime import datetime
time = datetime.strptime("2023-08-01 00:01:00", "%Y-%m-%d %H:%M:%S")




In [19]:
trajectories.iloc[0]["trajectory"]

TGeogPointSeq([POINT(12.606042 55.684293)@2023-08-01 00:00:00+02, POINT(12.606075 55.684307)@2023-08-01 00:00:09+02, POINT(12.606072 55.684335)@2023-08-01 00:00:19+02, POINT(12.606053 55.68442)@2023-08-01 00:00:29+02, POINT(12.606053 55.684463)@2023-08-01 00:00:38+02, POINT(12.606052 55.684472)@2023-08-01 00:01:00+02, POINT(12.606042 55.684465)@2023-08-01 00:01:09+02, POINT(12.60603 55.68445)@2023-08-01 00:01:18+02])

In [24]:
val = trajectories.iloc[0]["trajectory"].value_at_timestamp(time)
#coordinates of val 

print(val)

POINT (12.606052 55.684472)


In [27]:
trajectories

,trajectory,sog,distance
mmsi,,,
5322,[POINT(12.606042 55.684293)@2023-08-01 00:00:0...,"[0.1@2023-08-01 00:00:00+02, 0.2@2023-08-01 00...",23.883298
2190045,[POINT(8.423332 55.47179)@2023-08-01 00:00:07+...,"[0@2023-08-01 00:00:07+02, 0@2023-08-01 00:01:...",1.416454
109080297,[POINT(9.916937 57.815327)@2023-08-01 00:00:15...,"[1.9@2023-08-01 00:00:15+02, 1.9@2023-08-01 00...",0.000000
109080632,[POINT(7.840475 56.994502)@2023-08-01 00:00:59...,[0.9@2023-08-01 00:00:59+02],0.000000
205576930,[POINT(9.749615 54.971258)@2023-08-01 00:01:12...,[0.1@2023-08-01 00:01:12+02],0.000000
...,...,...,...
636023064,[POINT(8.830263 57.759765)@2023-08-01 00:00:01...,"[7.1@2023-08-01 00:00:01+02, 7.1@2023-08-01 00...",298.346515
636092635,[POINT(10.236577 56.15168)@2023-08-01 00:01:07...,[0@2023-08-01 00:01:07+02],0.000000
671057300,[POINT(10.813455 55.298015)@2023-08-01 00:00:4...,[0@2023-08-01 00:00:46+02],0.000000


In [28]:
#loop over all rows of trajectories 
for index, row in trajectories.iterrows():
    print(row["trajectory"].value_at_timestamp(time))
    print(row["sog"].value_at_timestamp(time))

POINT (12.606052 55.684472)
0.1
POINT (8.423327363636389 55.47179627272727)
0.0


TypeError: 'NoneType' object is not subscriptable